In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

c:\Users\NgLaam\Desktop\chatbot\medical-chatbot\medical-chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = '8d7fd4ae-327c-4cd5-abbe-0c2d4769e37e'
PINECONE_API_HOST = 'https://medical-chatbot-beutxtz.svc.gcp-starter.pinecone.io'

In [11]:
# Thiết lập biến môi trường cho Pinecone API key
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [3]:
#Extract PDF data
data_path = 'dataset/'

def load_pdf(data_path):
    loader = DirectoryLoader(data_path,
                        glob = '*.pdf',
                        loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

extract_data = load_pdf(data_path)

In [4]:
# extract_data

In [5]:
#Create text chunks
def text_split(extract_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extract_data)

    return text_chunks

text_chunks = text_split(extract_data)
print('Length of my chunk is ', len(text_chunks))

Length of my chunk is  7020


In [6]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

In [7]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
query = embeddings.embed_query('Good morning')
print('Length', len(query))

Length 384


In [12]:
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

index_name ="medical-chatbot"

#Creating Embeddings for each of the text chunks and storing 
docsearch = PineconeStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [18]:
#Load index if we already have index
docsearch = PineconeStore.from_existing_index(index_name, embeddings)

query = 'Tôi là ai'

docs = docsearch.similarity_search(query, k=3)
print('Result', docs)

Result [Document(page_content='body, including the muscles, heart, nerves, and digestivesystem. Beriberi literally means “I can’t, I can’t” in Sing-halese, which reflects the crippling effect it has on itsvictims. It is common in parts of southeast Asia, wherewhite rice is the main food. In the United States, beriberiis primarily seen in people with chronic alcoholism .\nDescription\nBeriberi puzzled medical experts for years as it rav-'), Document(page_content='namely C. argentinense ,C. butyricum , and C. baratii ,b u t'), Document(page_content='Introduction .................................................... ix\nAdvisory Board .............................................. xi\nContributors ................................................. xiii\nEntries\nVolume 1: A-B .............................................. 1\nVolume 2: C-F .......................................... 625\nVolume 3: G-M ....................................... 1375\nVolume 4: N-S ................................

In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
chain_type_kwargs={'prompt': PROMPT}

In [23]:
llm=CTransformers(model='model/llama-2-7b-chat.ggmlv3.q4_0.bin',
                  model_type='llama',
                  config={'max_new_tokens': 512,
                          'temperature':0.8})

In [24]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [25]:
while True:
    user_input=input(f'Input Prompt:')
    result=qa({'query': user_input})
    print('Response :', result['result'])

c:\Users\NgLaam\Desktop\chatbot\medical-chatbot\medical-chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response : The amount of phlegm brought up during a cough can vary consid-erably from person to person. It is not possible to say exactly how muchphlegm someone with chronic bronchitis will cough up without knowingthe individual.
